---

### 🎓 **Professor**: Apostolos Filippas

### 📘 **Class**: AI Engineering

### 📋 **Topic**: You Can Just Build Things

🚫 **Note**: You are not allowed to share the contents of this notebook with anyone outside this class without written permission by the professor.

---

## Welcome!

In our firstfour lectures, we've covered how
1. We can call LLMs via APIs and get structured responses
2. We can build lexical search with BM25
3. We can build semantic search with embeddings
4. We can combine lexical and semantic search into hybrid search

Today you will put it all together by building a Retrieval Augmented Generation (RAG) system.
- This is a question-answering bot that can answer questions about Fordham University
- You will use real data scraped from the Fordham website.


Your RAG pipeline will look like this:

```
User Question
     ↓
1. RETRIEVE: Find relevant documents (search!)
     ↓
2. AUGMENT: Stuff those documents into a prompt
     ↓
3. GENERATE: Ask an LLM to answer using the context
     ↓
Answer
```


---

# 1. Look at your data

In `data/fordham-website.zip` you'll find **~9,500 Markdown files** scraped from Fordham's website. Each file is one page — admissions info, program descriptions, faculty pages, financial aid, campus life, and more.

Your task: **look at the data**
- The first step in any AI engineering or data science project should always be to familiarize yourself with the data.
- I cannot stress this enough.. without this step, it's hard to build anything useful.

Tips:
- Unzip the archive and look at some of the files. 
- Open a few in a text editor. 
- Get a feel for what you're working with.
- The first line of every file is always the **URL** of the page it was scraped from. The rest is the page content converted to Markdown. Here's an example — `gabelli-school-of-business_veterans.md`:

```markdown
https://www.fordham.edu/gabelli-school-of-business/veterans

# Military Veterans & Active Duty Members of the Military

## Transform Your Knowledge & Skills Into a Business Career for the Future

As a veteran or an active duty member of the United States Armed Services,
you have gained or are currently acquiring the invaluable organizational,
leadership, analytics, and technical knowledge and skills that hiring
managers seek. These transferrable skills provide a major advantage in
emerging, business-related industries where innovation, a global mind-set,
and the ability to lead individuals and teams in the continuously evolving
work environment, are critical for success.

By completing a graduate or undergraduate business degree at the Gabelli
School of Business, you can prepare for a lifelong career in some of
today's fastest-growing fields. ...

### Study at a Top-Ranked, Military-Friendly University

The Gabelli School of Business is part of Fordham University, the only
New York City university to be among those ranked "Best for Vets" by
Military Times. ...

### Learn How the Yellow Ribbon Program Works

The Yellow Ribbon GI Education Enhancement Program, or the Yellow Ribbon
Program, is a part of the Post-9/11 Veterans Educational Assistance Act
of 2008. ...
```

The filenames mirror the URL structure — underscores replace path separators (e.g. `gabelli-school-of-business_veterans.md` came from `/gabelli-school-of-business/veterans`). Some files are short (a few lines), others are quite long.

- Once you've looked around, load the files into Python. Python's built-in `zipfile` module can read zip archives without extracting to disk. Load them into a list of dictionaries or a DataFrame with at least two fields: the filename (or a clean page name) and the content

In [1]:
# Placeholder for your implementation
import zipfile
from pathlib import Path
import pandas as pd
import json 


zip_path = '../data/fordham-website-windows.zip'
map_path = '../data/fordham-website-windows.json'

with open(map_path, "r", encoding="utf-8") as f:
    name_map = json.load(f)

rows = []
with zipfile.ZipFile(zip_path, "r") as zf:
    for name in zf.namelist():
        with zf.open(name) as f:
            content = f.read().decode("utf-8", errors="replace")
        
        clean_name = name_map.get(name)

        rows.append({
            "page_name": clean_name,
            "content": content
        })

df = pd.DataFrame(rows)

In [2]:
df.head()

,page_name,content
0,index.md,https://www.fordham.edu/\n\n## Doing Good That...
1,research.md,https://www.fordham.edu/research\n\nUncovering...
2,ccel.md,https://www.fordham.edu/ccel\n\n# Center for C...
3,fordham-college-at-lincoln-center.md,https://www.fordham.edu/fordham-college-at-lin...
4,academics.md,https://www.fordham.edu/academics\n\n# Academi...


---

# 2. Chunk the Documents

Some of the pages could be too long to embed as a single unit. Down the line, the pages may be too long to stuff into the LLM's prompt during the generation step. As such, most of the RAG systems will break down big documents into into smaller **chunks**.

> 📚 **TERM: Chunking**  
> Splitting documents into smaller, self-contained pieces for embedding and retrieval. The goal is chunks that are small enough to be specific, but large enough to be meaningful.

Your task: **write a function that splits each document into chunks.**

Things to think about:
- What's a reasonable chunk size? (Think about what fits in a prompt vs. what's too vague)
- Should you split on sentences? Paragraphs? A fixed character/word count?
- Should chunks overlap? What happens if an answer spans two chunks?
- How do you keep track of which document each chunk came from? You may need that information down the line.

In [3]:
import tiktoken

def chunk_by_tokens(text, max_tokens = 400, overlap = 50, model = 'text-embedding-3-small'):

    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)

    chunks = []
    start = 0

    step = max_tokens - overlap

    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_text = encoding.decode(tokens[start:end]).strip()
        if chunk_text:
            chunks.append(chunk_text)
        start += step
    
    return chunks

def build_chunk_records(df, embed_model = 'text-embedding-3-small', max_tokens=400, overlap=50):
    records = []
    for row_i, row in df.iterrows():
        page_name = row["page_name"]
        content = row["content"]
        
        chunks = chunk_by_tokens(
            content,
            max_tokens=max_tokens,
            overlap=overlap,
            model=embed_model,
        )

        for idx, chunk in enumerate(chunks):
            records.append({
                "id": f"{page_name}::chunk_{idx}",
                "page_name": page_name,
                "chunk_index": idx,
                "text": chunk,
            })
    return records

chunk_records = build_chunk_records(df)

In [11]:
chunk_records

[{'id': 'index.md::chunk_0',
  'page_name': 'index.md',
  'chunk_index': 0,
  'text': 'https://www.fordham.edu/\n\n## Doing Good That Becomes Greater As The Jesuit University of New York\n\nWe’re located in New York City—driven by our Jesuit values and tackling today’s most pressing issues at the center of the world stage.\n\n## We Are Leaders, Dreamers, Achievers, And Doers\n\nWith sound hearts, strong minds, and the wisdom to take charge, generations of Rams have found what they have needed to grow—the opportunities, connections, and support of this community.\n\n## From Winding Elms to Bustling City Blocks\n\nWith residential campuses in the Bronx and Manhattan, as well as campuses in Westchester and London, Fordham provides endless opportunities to start working toward your career and building the life you want.\n\n\n## We’re Drawn to Where We’re Needed Most\n\nExplore how our values come to life: how Fordham’s students, faculty, and alumni contribute to society and make lives bett

---

# 3. Embed the Chunks

Now we need to turn each chunk into a vector so we can search over them. You've done this before in Lecture 4.

Your task: **embed all chunks using an embedding model.**

Tips:
- You could use a local model, or API model. What are the tradeoffs?
- This will take a while if you do it serially. You might want to use async/batch.
- Once you've created your embeddings, you may want to save them to disk so you don't have to redo this step every time
- You'll need to embed queries with the **same model** at search time

In [7]:
import litellm
import numpy as np
from dotenv import load_dotenv
load_dotenv()
response = litellm.embedding(model="text-embedding-3-small", input=["couch"])

def embed_texts(texts, model = "text-embedding-3-small", batch_size = 128):
    vectors = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        resp = litellm.embedding(model=model, input=batch)
        vectors.extend([d["embedding"] for d in resp["data"]])

    return np.array(vectors, dtype=np.float32)

texts = [r["text"] for r in chunk_records]
emb = embed_texts(texts, model="text-embedding-3-small") 

emb = emb / (np.linalg.norm(emb, axis=1, keepdims=True) + 1e-12)

index = {
    "vectors": emb,
    "records": chunk_records,
}

In [10]:
index

{'vectors': array([[-0.05786267, -0.02527027,  0.00225468, ..., -0.00308862,
          0.0081194 ,  0.01021144],
        [ 0.03624962,  0.05656588,  0.04424052, ..., -0.00435565,
         -0.0119137 , -0.03109186],
        [-0.03994465, -0.00183098,  0.01432006, ..., -0.0111443 ,
         -0.02290043,  0.01943332],
        ...,
        [-0.02394276, -0.01716371,  0.0436872 , ..., -0.01716371,
         -0.04346494,  0.00501637],
        [-0.02172243, -0.01721982,  0.02693528, ..., -0.04979677,
         -0.01719302,  0.02830215],
        [-0.04415377, -0.02301492,  0.02948869, ..., -0.01531245,
         -0.01257761,  0.00052228]], shape=(38768, 1536), dtype=float32),
 'records': [{'id': 'index.md::chunk_0',
   'page_name': 'index.md',
   'chunk_index': 0,
   'text': 'https://www.fordham.edu/\n\n## Doing Good That Becomes Greater As The Jesuit University of New York\n\nWe’re located in New York City—driven by our Jesuit values and tackling today’s most pressing issues at the center of the

---

# 4. Retrieve

Now build the **R** in RAG. Given a user's question, find the most relevant chunks.

Your task: **write a retrieval function that takes a question and returns the most relevant chunks.**

Tips:
- You can use lexical or semantic search or both!
- How many chunks should you retrieve? Too few and you might miss the answer; too many and you'll overwhelm the LLM (and pay more tokens)
- Try a few test questions and eyeball whether the retrieved chunks are relevant
- Try a few questions and see what comes back. For example:
  - "What programs does the Gabelli School of Business offer?"
  - "How do I apply for financial aid?"
  - "Where is Fordham's campus?"

In [13]:
# Your implementation here
import numpy as np
import litellm

def retrieve(question: str, index: dict, top_k: int = 5, model: str = "text-embedding-3-small"):
    resp = litellm.embedding(model=model, input=[question])
    q = np.array(resp["data"][0]["embedding"], dtype=np.float32)
    q = q / (np.linalg.norm(q) + 1e-12)

    sims = index["vectors"] @ q
    top_idx = np.argsort(-sims)[:top_k]

    results = []
    for i in top_idx:
        rec = index["records"][int(i)]
        results.append({
            "score": float(sims[int(i)]),
            **rec,
        })
    
    return results

In [16]:
hits = retrieve("What programs does the Gabelli School of Business offer?", index, top_k=5)
for h in hits:
    print(h["text"][:250])

https://www.fordham.edu/info/20447/gabelli_school_of_business

# Gabelli School of Business

## Graduate Business Programs – Advance Your Career

The Gabelli School’s nationally and globally ranked graduate business degrees focus on leveraging busine
https://www.fordham.edu/gabelli-school-of-business/academic-programs-and-admissions

# Gabelli Academic Programs and Admissions

Our wide-ranging business school programs won’t just shape your career – they’ll offer a transformative experience that w
elli-school-of-business/academic-programs-and-admissions/graduate-programs/academic-programs/ms-programs/management/)(On-site and online. This program is not available to undergraduates in the Gabelli School of Business.)[Marketing Intelligence](/gab
https://www.fordham.edu/gabelli-school-of-business

# Gabelli School of Business

## Graduate Business Programs – Advance Your Career

The Gabelli School’s nationally and globally ranked graduate business degrees focus on leveraging business as a 

---

# 5. Generate

Now build the **G** in RAG. Take the retrieved chunks and pass them to an LLM along with the user's question.

Your task: **write a function that takes a question and the retrieved chunks, builds a prompt, and calls an LLM to generate an answer.**

Tips:
- How should you structure the prompt? The LLM needs to know: (1) what is the context of the application, (2) what is the question, (3) what it should include in its answer
- What should the LLM do if the context doesn't contain the answer?
- Start with a cheap model; try a better one when you've figured out the pipeline

In [ ]:
# Placeholder for your implementation
import litellm
def generation(query, chunks):
    context = ''.join(chunks)
    resp = litellm.acompletion(
        model = 'gpt-5-mini',
        messages = [{
                'role': 'system',
                'content': 'Generate answers based on the query and the retrieved information'
            },
            {
                'role': 'user',
                'content': f'This is the query that user inputs: {query}./nThis is the related context that we retrieved from knowledge base: {context}. If the context doesn\'t contain the answer, just output None.'
            }],
    )

    return resp.choices[0].message.content


---

# 6. Wire everything together

Combine the previous steps into a simple function that takes in a question and returns an answer.

Your task: **write a `rag(question)` function that retrieves relevant chunks and generates an answer.**

In [ ]:
# Placeholder for your implementation


---

# 7. Evaluate, experiment and improve

Your RAG system works — but there's always room to make it better. 

Your task: **evaluate, experiment, and improve your system**

Tips:
- How do you know that your system is working or that your changes are improving it?
- Try different questions — where does it do well? Where does it struggle?
- Adjust the number of retrieved chunks — what happens with more or fewer?
- Try different chunking strategies — bigger chunks? Smaller? Overlap?
- Try a different embedding model — does it change retrieval quality?
- Improve the prompt — can you get better, more concise answers?
- Add source attribution — can the system tell the user which pages the answer came from?

In [ ]:
# Placeholder for your implementation

---

# 8. (Optional) Make it an app

So far your RAG system lives inside a notebook. That's great for development — but nobody is going to use your Jupyter notebook to ask questions about Fordham. Let's turn it into a real web app.

> 📚 **TERM: Streamlit**  
> A Python library that turns plain Python scripts into interactive web apps. You write Python — no HTML, CSS, or JavaScript — and Streamlit renders it as a web page with inputs, buttons, and formatted output. It's the fastest way to go from "I have a function" to "I have a web app."

Your task: **create a Streamlit app that lets a user type a question about Fordham and get an answer from your RAG system.**

To get started:
- Install it: `uv pip install streamlit` 
- A Streamlit app is just a `.py` file (not a notebook). Create something like `fordham_rag_app.py`
- Run it: `streamlit run scripts/fordham_rag_app.py` — this opens a browser tab with your app

Tips:
- Check out the [Streamlit docs](https://docs.streamlit.io/) — the "Get started" tutorial is very short
- Your best bet is to vibecode your way to this. You'll be surprised how fast you can get it up and running

---

# Summary

## What You Built

| Step | What You Did | What It Does |
|------|-------------|-------------|
| **Load** | Read 9,500+ Fordham web pages | Get raw content |
| **Chunk** | Split pages into smaller pieces | Make content searchable and promptable |
| **Embed** | Turn chunks into vectors | Enable semantic search |
| **Retrieve** | Find relevant chunks for a question | The **R** in RAG |
| **Generate** | Ask an LLM to answer using the chunks | The **G** in RAG |
| **RAG** | Wire it all together | Question in, answer out |

## The Big Picture

RAG is one of the most common patterns in AI engineering today. What you built here is the same core architecture behind tools like ChatGPT with search, Perplexity, enterprise Q&A bots, and more. The details get more sophisticated (vector databases, reranking, query rewriting, evaluation) but the pattern is the same:

**Find relevant stuff → give it to an LLM → get an answer.**

You can just build things.